## The problem: Rod cutting

INPUT: $P$, $n$.

$P$ is an array of nonnegative integers such that $P[0]=0$. $n$ is an integer such that $1\le n<\mathrm{len}(P)$.

OUTPUT:

Rod of length $i$ can be sold for price $P[i]$. Cut a longer rod of length $n$ into a number of pieces so that the total price is maximized.

EXAMPLE:

For $P = [0, 2, 5, 6, 9, 12, 13, 13, 15]$, $n=6$, the optimal value is $15=3P[2]$ (cut the rod of length 6 into 3 pieces of length 2 each).

## Recursive solution

For a given $P$, let $T(n)$ denote the optimal solution for a rod of length $n$. Then, we have
$T(0)=0$. For $n>0$, we have:

$$T(n)=\max\Big(P[n],\max_{i=1}^{n-1} \left\{T(i)+T(n-i)\right\}\Big)$$

This is because in optimal solution for $n$:
* Either there is zero cuts, which means the value is $P[n]$.
* Or there is at least one cut. This cut can divide $n$ into two parts $i$ and $n-i$ for some $i=1,2,\ldots,n-1$. The optimal solution that contains a cut at position $i$ must consist of optimal solution for $i$ and optimal solution for $n-i$. Then, its total cost is $T(i)+T(n-i)$.

Therefore, the recursive formula takes into account all possible solutions.

The formula can also be changed to:

$$T(n)=\max\Big(P[n],\max_{i=1}^{n-1} \left\{T(i)+P[n-i]\right\}\Big)$$

The justification is similar:
* With zero cuts, the value is $P[n]$.
* If there is a cut, consider the **rightmost** cut in the optimal solution. This cut has position $i$ for some $1\le i\le n-1$. Then, the solution has exactly one piece to the right of the cut (value $P[n-i]$) and more pieces on the left with optimal value $T(i)$.

## Rod cutting: Recursive solution

In [ ]:
example_P = [0, 2, 5, 6, 9, 12, 13, 13, 15]
len(example_P)

In [ ]:
# This solution is wrong, because it does not use the recursive formula correctly.

def rod_cutting_incorrect(P, n):
    s = P[n]
    for i in range(1, n):
        s = max(s, P[i]+P[n-i])
    return s

In [ ]:
rod_cutting_incorrect(example_P, 6)

In [ ]:
# This is a correct implementation with recursion.

def rod_cutting(P, n):
    s = P[n]
    for i in range(1, n):
        s = max(s, rod_cutting(P, i)+rod_cutting(P, n-i))
    return s

In [ ]:
rod_cutting(example_P, 6)

In [ ]:
# Also the second formula will give the same results. Since we are calling recursion once, not twice, the runtime can be reduced.

def rod_cutting_one_recursion(P, n):
    s = P[n]
    for i in range(1, n):
        s = max(s, rod_cutting_one_recursion(P, i)+P[n-i])
    return s

In [ ]:
rod_cutting_one_recursion(example_P, 6)

## Recursion time complexity

Let us try larger arrays.

In [ ]:
large_P = example_P + [20]*300
len(large_P)

In [ ]:
import time
start = time.time()
rod_cutting(large_P, 16)
end = time.time()
print('%.2f seconds' % (end-start))

Version with one recursion is faster, but only somewhat faster:

In [ ]:
start = time.time()
rod_cutting_one_recursion(large_P, 16)
end = time.time()
print('%.2f seconds' % (end-start))

In [ ]:
start = time.time()
rod_cutting_one_recursion(large_P, 24)
end = time.time()
print('%.2f seconds' % (end-start))

## Exercise

Consider a function $K:\mathbb{N}\to\mathbb{N}$ given as $K(1)=1$ and
$K(n)=1+\sum_{i=1}^{n-1} K(i)$. Prove by induction that $K(n)=2^n$.

Note that `rod_cutting_one_recursion(n)` calls itself on the arguments from $1$ to $n-1$. By the exercise, its time complexity must be at least $\Omega(2^n)$. This is called **exponential time** and is not practical for large values of $n$.

## Improving the recursion

The reason for slow performance of the recursion is that many computations are unnecessarily repeated. For every $i$, the call to `rod_cutting(i)` should give the same result. But now, the whole recursion is performed each time `rod_cutting(i)` is called. 

In [ ]:
def rod_cutting_print(P, n):
    print(n, end=' ')
    s = P[n]
    for i in range(1, n):
        s = max(s, rod_cutting_print(P, i)+P[n-i])
    return s

In [ ]:
rod_cutting_print(example_P, 8)

## Solution 1: Memoization

In [ ]:
# This technique is called memoization. Remember if you already computed the subproblem. 
# If yes, immediately return the remembered value.

def rod_cutting_memoization(P, n):
    T = [-1]*(n+1)
    return rod_cutting_rec(P, T, n)
def rod_cutting_rec(P, T, n):
    if T[n] != -1:   # Check if the subproblem already computed. If yes, return remembered value.
        return T[n]
    s = P[n]
    for i in range(1, n):
        s = max(s, rod_cutting_rec(P, T, i)+P[n-i])
    T[n] = s        # Remember the computed value for later
    return s

In [ ]:
rod_cutting_memoization(example_P, 6)

In [ ]:
rod_cutting_memoization(large_P, 200)

## Solution 2: Eliminate recursion

In [ ]:
# T[n] contains the solution for problem of size n.

def rod_cutting_iter(P, n):
    T = [0]*(n+1)
    for i in range(1, n+1):
        s = P[i]
        for j in range(1, i):
            s = max(s, P[j]+T[i-j])
        T[i] = s
    return T[n]

In [ ]:
example_P

In [ ]:
rod_cutting_iter(example_P, 6)

In [ ]:
rod_cutting_iter(large_P, 200)

Both memoization and iterative solution run in total time $O(n^2)$.

## Outputting complete solution

So far the procedure `rod_cutting_iter` only outputs the price of the optimal cutting. Below is a modified procedure which also outputs the list of cuts to be made in the optimal solution.

In the list `first_cut[i]` we store what should be the first cut in the optimal solution for `i`. If `first_cut[i]==[i]`, then we do not cut the rod. Otherwise, we cut off a piece of length `first_cut[i]`. The remaining rod
has length `i-first_cut[i]` and it should also be cut optimally. Accordingly, we substitute `i := i-first_cut[i]` and repeat the procedure. This can be repeated multiple times (using a `while` loop) as necessary.

In [ ]:
def rod_cutting_complete(P, n):
    T = [0]*(n+1)
    first_cut = [0]*(n+1)
    for i in range(1, n+1):
        s = P[i]
        index_of_best_cut = i
        for j in range(1, i):
            if P[j]+T[i-j] > s:
                s = P[j]+T[i-j]
                index_of_best_cut = j
        T[i] = s
        first_cut[i] = index_of_best_cut

    result = []
    remaining = n
    while remaining > 0:
        c = first_cut[remaining]
        result.append(c)
        remaining -= c
    
    return T[n], result

In [ ]:
rod_cutting_complete(example_P, 6)